<!-- RGB Model -->
<div class="alert" style="background: linear-gradient(to right,rgb(255, 0, 0), rgb(0,255,0),rgb(0, 0, 255)); 
color:rgb(255, 255, 255);">

# **U-NET RGB Colorization Model - V2**
***
### **U-NET RGB Colorization Model.**


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../Src')
from unet_model import build_unet, get_callbacks
import tensorflow as tf

from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model

In [3]:
# GPU Check & Mixed Precision Setup for TensorFlow
import tensorflow as tf
from tensorflow.python.client import device_lib

print(f"TensorFlow version: {tf.__version__}")
print(f"Built with CUDA: {tf.test.is_built_with_cuda()}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

print("Device List:")
for device in device_lib.list_local_devices():
    print(f" - {device.name} ({device.device_type})")

# Enable mixed precision for performance if GPU is available
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        from tensorflow.keras import mixed_precision
        mixed_precision.set_global_policy("mixed_float16")
        print("Mixed precision enabled (float16).")
    except Exception as e:
        print(f"Could not enable mixed precision: {e}")
else:
    print("No GPU detected. Running on CPU.")

TensorFlow version: 2.10.0
Built with CUDA: True
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Device List:
 - /device:CPU:0 (CPU)
 - /device:GPU:0 (GPU)
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3080, compute capability 8.6
Mixed precision enabled (float16).


In [4]:
import sys
print(sys.executable)


c:\Users\sebas\Desktop\PythonMetaRepo\CNN_U-Net_GAN_Project\.venv\Scripts\python.exe


In [5]:
X_train = np.load("../Data/prepared_data/RGB/comic_input_grayscale_train.npy")
print(X_train.dtype)

float32


In [6]:
x = np.load("../Data/prepared_data/RGB/comic_input_grayscale_train.npy", mmap_mode=None)
print("Shape:", x.shape)
print("Min/max:", np.min(x), np.max(x))
print("Type before:", x.dtype)

x = x.astype(np.float32)
print("Type after:", x.dtype)

x = (x - 0.5) * 2.0
print("Final min/max:", np.min(x), np.max(x))

Shape: (256, 256, 256, 1)
Min/max: -1.0 1.0
Type before: float32
Type after: float32
Final min/max: -3.0 1.0


In [7]:
import tensorflow as tf
print(tf.__version__)

2.10.0


In [8]:
# Load and preprocess data
X_train = np.load("../Data/prepared_data/RGB/comic_input_grayscale_train.npy")
y_train = np.load("../Data/prepared_data/RGB/comic_output_color_train.npy")
X_test  = np.load("../Data/prepared_data/RGB/comic_input_grayscale_test.npy")
y_test  = np.load("../Data/prepared_data/RGB/comic_output_color_test.npy")

# (Optional safety cast)
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

X_train: (256, 256, 256, 1)
y_train: (256, 256, 256, 3)
X_test: (64, 256, 256, 1)
y_test: (64, 256, 256, 3)


NameError: name 'model' is not defined

In [ ]:
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

print("Model input:", model.input_shape)
print("Model output:", model.output_shape)


In [ ]:
# Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[checkpoint_cb, earlystop_cb]
)

model.save("U-NET_RGB_final_trained_v2.keras")
print("Saved best and final model.")


Epoch 1/100


: 

In [ ]:
# Visualize some results
def plot_results(model, X, y, num_samples=50):
    predictions = model.predict(X[:num_samples])
    
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 5*num_samples))
    titles = ['Grayscale Input', 'Predicted Color', 'Ground Truth']
    
    for i in range(num_samples):
        axes[i, 0].imshow(X[i].squeeze(), cmap='gray')
        axes[i, 1].imshow(predictions[i])
        axes[i, 2].imshow(y[i])
        
        for j in range(3):
            axes[i, j].axis('off')
            if i == 0:
                axes[i, j].set_title(titles[j])
    
    plt.tight_layout()
    plt.show()

# Plot test results
plot_results(model, X_test, y_test)

In [ ]:
# Check variables
%whos